# 05 — Autoencoder Feature Learning

In [ ]:

# Update this if your data isn't under ./data
base_path = r"./data"  # change to r"D:\IITB\STData" on Windows if needed
save_models_to = r"./models"
save_fig_to = r"./notebooks/figures"

import os, pandas as pd, numpy as np, matplotlib.pyplot as plt
os.makedirs(save_models_to, exist_ok=True)
os.makedirs(save_fig_to, exist_ok=True)

def read_csv(name):
    p = os.path.join(base_path, name)
    return pd.read_csv(p)

print("Using base_path:", base_path)


In [ ]:

import os, torch, torch.nn as nn, torch.optim as optim, pandas as pd, numpy as np

X = pd.read_csv(os.path.join(base_path,"processed_clean.csv")).values.astype("float32")
X = torch.tensor(X)

class AE(nn.Module):
    def __init__(self, d_in= X.shape[1], d_lat=8):
        super().__init__()
        self.enc = nn.Sequential(nn.Linear(d_in,128), nn.ReLU(), nn.Linear(128,64), nn.ReLU(), nn.Linear(64,d_lat))
        self.dec = nn.Sequential(nn.Linear(d_lat,64), nn.ReLU(), nn.Linear(64,128), nn.ReLU(), nn.Linear(128,d_in))
    def forward(self, x): 
        z = self.enc(x); xh = self.dec(z); 
        return xh, z

model = AE()
opt = optim.Adam(model.parameters(), lr=1e-3)
lossf = nn.MSELoss()

for epoch in range(50):
    opt.zero_grad(); xh, z = model(X); loss = lossf(xh, X); loss.backward(); opt.step()
    if (epoch+1)%10==0: print("epoch", epoch+1, "loss", float(loss))

torch.save(model.state_dict(), os.path.join(save_models_to,"autoencoder_model.pt"))
print("Saved autoencoder weights")

# Save latent for plotting in next notebook if needed
with torch.no_grad():
    _, Z = model(X)
    np.savetxt(os.path.join(base_path,"autoencoder_latent.csv"), Z.numpy(), delimiter=",")
